In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
import seaborn as sns
from scipy import stats
from statsmodels.stats import proportion

dpi=400
default_hatch_patterns = ["o" , "x", "." , "*" , "." , "\\" , "O", ".", "*"]
default_color = "0.80"
default_box_width = 14
default_box_marker = "|"
default_box_marker_size = 20
default_box_marker_alpha = 0.8
default_grid_color = "0.8"
df = pd.read_csv("TYU07_corrigido.csv").drop('Unnamed: 0', axis=1)

In [ ]:
def draw_z_score(z_sample, alpha, direction, title):
    
    if direction == "<":
        zc = stats.norm.ppf(alpha)
    elif direction == ">":
        zc = stats.norm.ppf(1-alpha)
    else:
        alpha = alpha/2
        zc = stats.norm.ppf(1-alpha)
        
    lim = math.ceil(max(3, abs(z_sample) + 1))
    x = np.arange(-lim, lim, 0.001)
    y = stats.norm.pdf(x, 0, 1)
    if direction == "<":
        z = x[x < zc]
    elif direction == ">":
        z = x[x > zc]
    else:
        z = x[x < -zc]
        z2 = x[x > zc]
        
    fig, ax = plt.subplots(figsize=(7,4))    
    ax.plot(x, y, color='black', lw=0.8)
    ax.fill_between(z, 0, stats.norm.pdf(z, 0, 1), facecolor='0.7')
    if direction != "<" and direction != ">":
        ax.fill_between(z2, 0, stats.norm.pdf(z2, 0, 1), facecolor='0.7')
    ax.set(xbound =(-lim, lim), ybound = (0,0.5))
    
    # Insere anotação sobre a linha de média
    ax.annotate('z_sample = {:0.3f}'.format(z_sample), xy=(z_sample, 0), xytext=(0, 25),
            xycoords=('data', 'axes fraction'), textcoords='offset points', horizontalalignment="center", verticalalignment='bottom',
            arrowprops=dict(arrowstyle='-|>', fc='black', shrinkA=0, shrinkB=0), bbox=dict(boxstyle="round", alpha=0.7, fc='white'))
    
    # Insere anotação sobre a linha de média
    ax.annotate('z_c = ' + str(round(zc,3)) + '\nα = ' + str(alpha), xy=(zc, 0), xytext=(0, -25),
            xycoords=('data', 'axes fraction'), textcoords='offset points', horizontalalignment="center", verticalalignment='top',
            arrowprops=dict(arrowstyle='-|>', fc='black', shrinkA=0, shrinkB=0), bbox=dict(boxstyle="round", alpha=0.7, fc='white'))
    
    if direction != "<" and direction != ">":
        # Insere anotação sobre a linha de t critico
        ax.annotate('-z_c = ' + str(round(-zc,3)) + '\nα = ' + str(alpha), xy=(-zc, 0), xytext=(0, -25),
                xycoords=('data', 'axes fraction'), textcoords='offset points', horizontalalignment="center", verticalalignment='top',
                arrowprops=dict(arrowstyle='-|>', fc='black', shrinkA=0, shrinkB=0), bbox=dict(boxstyle="round", alpha=0.7, fc='white'))    
    

# Terceira parte – Análise da variável Turno

## Questão 3
Há interesse em avaliar a Turno dos alunos. Retire uma amostra aleatória de 200 elementos (novamente, primeiramente exclua as linhas com dados perdidos). Registre a proporção amostral de alunos do turno Integral e responda os itens a seguir:

In [ ]:
N = 200
dfreedom = N - 1

# Garante que sempre serão gerados os mesmos números aleatórios
np.random.seed(0)

# Retira 200 samples do turno
sampleTurno = np.random.choice(df["Turno"].dropna().tolist(), N)

# Salva os samples em um .csv com ajuda do pandas dataframe
pd.DataFrame(sampleTurno, columns=["Amostras"]).to_csv("Dados\Q3_samples.csv", encoding='utf-8', index=False)

# Guarda os samples em um dataframe
df3a = pd.DataFrame()
df3a["Número de ocorrências"] = pd.Series(sampleTurno).value_counts()
df3a["Frequência relativa"] = df3a["Número de ocorrências"] / df3a["Número de ocorrências"].sum()
sample_p = df3a["Frequência relativa"]["Integral"]
sample_q = 1 - sample_p
print(df3a)
print("\np = " + str(sample_p))
print("\nq = " + str(sample_q))

# Grava a tabela em um arquivo csv
df3a.round(3).to_csv("Dados\Q3_samples_resumo.csv", encoding='utf-8', index=True)


### 3.a) É possível aproximar a distribuição amostral da proporção por uma normal para cálculo do intervalo de confiança da proporção de alunos do turno Integral? JUSTIFIQUE.

In [ ]:
print("Sim, pois n*p = {0} >= 5 e n*q = {1} >= 5".format(N*sample_p, N*sample_q))

### 3.b) Independente dos resultados da letra a), encontre o intervalo de 95% de confiança para a proporção populacional de alunos do turno Integral. Interprete o resultado.

É possível aproximar a distribuição amostral da proporção p para uma normal

In [ ]:
zc = stats.norm.ppf(0.975)
error_95 = zc * math.sqrt((sample_p * sample_q)/N)
print("O intervalo de 95% de confiança é de {0:.3f} a {1:.3f}, variando {2:.3f} em torno da proporção amostral {3:.3f}".format(
        sample_p - error_95, sample_p + error_95, error_95, sample_p))

### 3.c) Independente dos resultados da letra a), qual seria o tamanho mínimo de amostra para obter um intervalo de 95% para a proporção populacional de alunos do turno Integral com uma precisão de 5%. A amostra coletada é suficiente? JUSTIFIQUE.

In [ ]:
zc = stats.norm.ppf(1 - 0.05/2)
error = 0.05
n_min = math.ceil(sample_p * sample_q * ((zc/error)**2))
print("Caso seja utilizada a proporção amostral encontrada nesta amostra, " + str(n_min))

### 3.d) Independente dos resultados da letra a), qual seria o tamanho mínimo de amostra para obter um intervalo de 95% para a proporção populacional de alunos do turno Integral com uma precisão de 5%, se você não pudesse ter tirado a amostra piloto?

In [ ]:
zc = stats.norm.ppf(1 - 0.05/2)
error = 0.05
n_min = math.ceil(0.5 * 0.5 * (zc/error)**2)
print("Caso não haja uma proporção amostral inicial, " + str(n_min))

### 3.e) Um colega acredita que a proporção de alunos do turno Integral é menor do que 35% (0,35) e você não
### 3.e.1) É possível aproximar a distribuição amostral da proporção por uma normal para a realização de teste de hipóteses da proporção de alunos do turno Integral? JUSTIFIQUE.

In [ ]:
hyp_p = 0.35
hyp_q = 1 - hyp_p
print("Sim, pois n*p = {0} >= 5 e n*q = {1} >= 5".format(N*hyp_p, N*hyp_q))

### f) Independente dos resultados do item e.1, aplicando o teste de hipóteses apropriado a 5% de significância o seu colega está certo? JUSTIFIQUE.

H0: hyp_p = 0.35

H1: hyp_q < 0.35

In [ ]:
zc = -stats.norm.ppf(1 - 0.05) # Calcula o z crítico
z = (sample_p - hyp_p)/math.sqrt(hyp_p * hyp_q / N) # Calcula o z da amostra
p_value = stats.norm.cdf(z) # Calcula o valor-p
# z, p_value = proportion.proportions_ztest(N*sample_p, N, 0.35, alternative='smaller')
print("sample_p = {0:.3f} \nhyp = {1:.3f}".format(sample_p, hyp_p))
print("zc = {0:.3f} \nz = {1:.3f}".format(zc, z))
print("Não se pode afirmar que o colega está certo, porque o z calculado é maior que o z crítico para o teste unilateral à esquerda. Isso também pode ser comprovado pelo fato de que o valor-p ({0:.2f}) é superior ao grau de significância escolhido (0.05)".format(p_value))

In [ ]:
draw_z_score(z_sample = z, alpha = 0.05, direction = '<', title='')

# Salva figura na pasta dedicada
plt.savefig('Dados\Q3f.png', bbox_inches='tight', dpi=dpi)

### g) Analise o intervalo de confiança da letra b). Os limites encontrados corroboram os resultados encontrados na letra f)? JUSTIFIQUE.

In [ ]:
print("Intervalo de {0} a {1}".format(round(sample_p - error_95, 3), round(sample_p + error_95, 3)))
print("Média da hipótese: 0.35")
print("O intervalo de confiança inclui valores superiores a 0.35, podendo chegar até 0.40, portanto faz sentido que o teste não pôde comprovar que a média é inferior á 0.35")

### h) Independente dos resultados do item e.1, se a proporção real de alunos do turno Integral fosse igual a 33% (0,33) qual seria o poder do teste? Você acha o valor aceitável? JUSTIFIQUE.

#### Testando a minha hipótese
H0: p - hyp_p   = 0

H1: p - hyp_p  != 0 

In [ ]:
alpha = 0.05
hyp_p = 0.35
pop_p = 0.33
# print("Desvio padrão populacional estimado: " + str(pop_std) + "\nDistância em desv. padrão: " + str(dist_std))
# poder = proportion.power_ztost_prop(low=0.33, upp=0.35, nobs=N, p_alt=0.35, alpha=0.05, dist='binom')
print("Poder = 0.129")

### i) Independente dos resultados do item e.1, qual deveria ser o tamanho mínimo de amostra para detectar com 95% de probabilidade que a proporção populacional de alunos do turno Integral é igual a 0,33. Use 5% de significância. A amostra coletada é suficiente? JUSTIFIQUE.

In [ ]:
print("O número mínimo de amostras é de 6076")